import libraries

In [1]:
import MetaTrader5 as mt5
import numpy as np  # to access historical data
import pandas as pd  # for data analysis and calculation of technical indicators
import plotly.express as px  # for data visualization
from IPython.display import display, Markdown, Latex  # to display results in Jupyter Notebook
from datetime import datetime, timezone, timedelta  # import datetime library to specify the datetime range for historical data
import talib as ta  # to calculate technical indicators
import mplfinance as mpf # to plot candlestick charts
from pytz import timezone # importing timezone from pytz module
import time
import pandas_ta as pta 

get portfolio details

In [2]:
def get_portfolio(account_info):
    print("------------------------------------------------------------------")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Username: {account_info.login},\t"
        f"Balance: {account_info.balance} USD,\t"
        f"Equity: {account_info.equity} USD, \t"
        f"Profit: {account_info.profit} USD")
    print("------------------------------------------------------------------")

initialize and login to the trading account

In [3]:
def initialize():
    if not mt5.initialize(): 
        print("initialize() failed, error code =",mt5.last_error())
        quit() 

    acc_login = 6253898
    acc_password = "6e6fRY512#tZ#"
    acc_server = "OANDA-Demo-1"

    mt5.login(acc_login, acc_password, acc_server)
    
    current_account_info = mt5.account_info()
    get_portfolio(current_account_info)

import ohlc data

In [4]:
def get_ohlc_data(symbol, timeframe, from_date, number_of_data):

    ohlc = mt5.copy_rates_from(symbol, timeframe, from_date, number_of_data)
    ohlc_df = pd.DataFrame(ohlc)[['time', 'open', 'high', 'low', 'close']]

    # time_format = "%d-%m-%Y %H:%M:%S %Z%z"  
    time_format = "%d-%m-%Y %H:%M:%S %Z"

    # ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s').dt.strftime(time_format)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s').dt.tz_localize('Asia/Bangkok').dt.strftime(time_format)
    
    # convert timezone to Thailand timezone UTC+7 from Weltrade seerver timezone UTC+2
    # ohlc_df['time'] = ohlc_df['time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok').dt.strftime(time_format)
    
    return ohlc_df

add rsi strategy

In [5]:
def rsi_strategy (df_rsi, rsi_period=7, ovb=70, ovs=30, mid=30):
    
    df_rsi['rsi'] = ta.RSI(df_rsi['close'], timeperiod=rsi_period)
    df_rsi['rsi_lag'] = df_rsi['rsi'].shift(1)    
    
    current_rsi = df_rsi['rsi']
    previous_rsi = df_rsi['rsi_lag']
    
    crossoverbought = np.where((current_rsi >= ovb) & (previous_rsi < ovb), 'yes', 'no')
    crossoversold = np.where((current_rsi <= ovs) & (previous_rsi > ovs), 'yes', 'no')  

    df_rsi['crossovb'] = crossoverbought
    df_rsi['crossovs'] = crossoversold
    
    exit_long = np.where((current_rsi <= mid) & (previous_rsi > mid), 'yes', 'no')
    exit_short = np.where((current_rsi >= mid) & (previous_rsi < mid), 'yes', 'no')

    df_rsi['exit_long'] = exit_long
    df_rsi['exit_short'] = exit_short
    
    df_rsi.loc[df_rsi['crossovb'] == 'yes', 'action'] = 'buy'
    df_rsi.loc[df_rsi['crossovs'] == 'yes', 'action'] = 'sell'
    df_rsi.loc[df_rsi['action'] == 'nan', 'action'] = float('nan')
    
    # display(df_rsi)
    
    return df_rsi

send order

In [6]:
def send_order(request):
    # Send the order
    ordered = mt5.order_send(request)
    if ordered.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Failed to Send the Order: {ordered.comment}")
    else:
        print(f"{request['comment']} Sent Successfully")
    
    return ordered

make request for order sending

In [7]:
def make_request(symbol, volume, order_type):
    
    order_type_dict = {
    'buy': mt5.ORDER_TYPE_BUY,
    'sell': mt5.ORDER_TYPE_SELL}

    price_dict = {
    'buy': mt5.symbol_info_tick(symbol).ask,
    'sell': mt5.symbol_info_tick(symbol).bid}
    
    '''
     stop loss:
        buy -> below curent bid price
        sell -> above current ask price
        
     take profit:
        buy -> above current bid price
        sell -> below current ask price
    '''
    
    # sl_dict = {
    #     'buy': price_dict['sell'] - input_sl,
    #     'sell': price_dict['buy'] + input_sl}
    
    # tp_dict = {
    #     'buy': price_dict['sell'] + input_tp,
    #     'sell': price_dict['buy'] - input_tp}
    
    comment_dict = {
        'buy': 'Buy Order',
        'sell': 'Sell Order'}
        
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        # "sl": sl_dict[order_type],
        # "tp": tp_dict[order_type],
        "comment": comment_dict[order_type],
        "magic": 123456,  # Optional magic number
    }
    
    send_order(request)

make order

In [8]:
def make_order(df,symbol,volume,position_size=mt5.positions_total()):
    global ordered
    
    trade_df = pd.DataFrame(columns=['time', 'action', 'price', 'profit'])
    past_index = df.index[0]
    
    for index in df.index:
        # check if there is a position
        while position_size != 0:
            if df.iloc[-1]['exit_long'] == 'yes':
                ordered = make_request(symbol, volume, 'sell')
                
                action_type = 'exit_long' # last row of exit_long column
                # action_type = 'exit long'
                action_time = df.time[index]
                # action_time = str(datetime.now())
                exit_price = df.close[index]
                price = exit_price
                distance = round(exit_price - entry_price, num_digits = 2)
                profit = round(volume * distance, num_digits = 4)
                
                print('exit long at : ' + action_time + ' ' + str(exit_price) + ' ' + str(profit) + ' ' + index + '\n')
                # log.append(f'exit long {symbol} at : ' + str(datetime.now()))
                # position_size = 0
                # break
            
            elif df.iloc[-1]['exit_short'] == 'yes':
                ordered = make_request(symbol, volume, 'buy')
                
                action_type = df.iloc[-1]['exit_short'] # last row of exit_short column
                # action_type = 'exit short'
                action_time = df.time[index]
                # action_time = str(datetime.now())
                exit_price = df.close[index]
                price = exit_price
                distance = round(exit_price - entry_price, num_digits = 2)
                profit = round(volume * distance, num_digits = 4)
                
                print('exit short at : '+ action_time + ' ' + str(exit_price) + ' ' + str(profit) + ' ' + index + '\n')
                # log.append(f'exit short {symbol} at : ' + str(datetime.now()))
                # position_size = 0
                # break
            
            else:
                action_type = np.nan
                action_time = df.time[index]
                # action_time = str(datetime.now())
                price = np.nan
                profit = np.nan
                print('no exit at : ' + action_time)
                # log.append(f'no exit at : ' + str(datetime.now()))
                # break
        
        # check if there is no position
        while position_size == 0:
            if df.iloc[-1]['action'] == 'buy':
                ordered = make_request(symbol, volume, 'buy')
                
                action_type = df.iloc[-1]['action'] # last row of action column
                # action_type = 'action buy'
                action_time = df.time[index]
                # action_time = str(datetime.now())
                entry_price = df.close[index]               
                price = entry_price
                distance = round(exit_price - entry_price, num_digits = 2)
                profit = round(volume * distance, num_digits = 4)
                
                print('place buy at : '+ action_time + ' ' + str(entry_price) + ' ' + str(profit) + ' ' + index + '\n')
                
                # log.append(f'buy {symbol} at : ' + str(datetime.now()))
                # position_size+1
                # break
            
            elif df.iloc[-1]['action'] == 'sell':
                ordered = make_request(symbol, volume, 'sell')
                
                action_type = df.iloc[-1]['action'] # last row of action column
                # action_type = 'action sell'
                action_time = df.time[index]
                # action_time = str(datetime.now())
                entry_price = df.close[index]
                price = entry_price
                distance = round(exit_price - entry_price, num_digits = 2)
                profit = round(volume * distance, num_digits = 4)
                
                print('place sell at : '+ action_time + ' ' + str(entry_price) + ' ' + str(profit) + ' ' + index + '\n')
                
                # log.append(f'sell {symbol} at : ' + str(datetime.now()))
                # position_size+1
                # break
            
            else:
                action_type = df.iloc[-1]['action'] # last row of action column
                action_time = df.time[index]
                # action_time = str(datetime.now())
                price = np.nan
                profit = np.nan
                print('no action at : ' + action_time)
                # log.append(f'no action at : ' + str(datetime.now()))
                # position_size = 0
                # break
        
        
        # trade_df = trade_df.append({'time': df.index[index], 'action': action_type, 'price': price, 'profit': profit}, ignore_index=True)
        
        
        new_row = pd.DataFrame({'index': [index], 'time': [action_time], 'action': [action_type], 'price': [price], 'profit': [profit]})
        # new_row = pd.DataFrame({'index': index, 'time': action_time, 'action': action_type, 'price': price, 'profit': profit})
        trade_df = pd.concat([trade_df, new_row], ignore_index=True)
        # trade_record = pd.concat([trade_record, new_row], Ignore_index = True)
        # # past_index = index
        # display(trade_record)
        # display(trade_df, trade_df.__len__())
            
    return trade_df

execution

In [9]:
def execute(symbol, lot, timeframe, from_date, number_of_data):
    # get ohlc data
    ohlc = get_ohlc_data(symbol, timeframe, from_date, number_of_data)
        
    # get rsi, rsi_lag, crossoverbought, crossoversold, exit_long, exit_short, action
    rsi_pattern = rsi_strategy(ohlc)
    
    # make order
    make_order(rsi_pattern, symbol, lot)

main

In [10]:
def main():    
    initialize()
    # symbol = 'EURUSD'
    symbol = 'EURUSD.sml'
    lot = 0.01
    timeframe = mt5.TIMEFRAME_H1
    from_date = datetime.now()
    # from_date = datetime(2023, 11, 25) # year,month,day
    number_of_data = 100 # for forex, not counting the weekends (saturday and sunday)

    execute(symbol, lot, timeframe, from_date, number_of_data)  
    
    # ohlc = get_ohlc_data(symbol, timeframe, from_date, number_of_data)
    # rsi_pattern = rsi_strategy(ohlc)
    # rsi_pattern

In [11]:
main() 

------------------------------------------------------------------
Date: 2024-02-15 14:34:40
Username: 6253898,	Balance: 9999.72 USD,	Equity: 9999.72 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-2024 04:00:00 +07
no exit at : 09-02-20

KeyboardInterrupt: 